<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#LinearSVC" data-toc-modified-id="LinearSVC-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LinearSVC</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, accuracy_score
import torch
import transformers
from tqdm import notebook
from tqdm import tqdm
from functools import lru_cache

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.shape

(159292, 3)

In [5]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

<b>Токсичных комментариев - 16186, не токсичных - 143106.</b>

<b>Лемматизируем данные</b>

In [6]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [7]:
stop_words = set(nltk_stopwords.words('english'))
@lru_cache(maxsize=128)
def get_wordnet_pos(treebank_tag):
    """Расширенное преобразование POS-тегов с учетом большего количества случаев"""
    tag_mapping = {
        'J': wordnet.ADJ,    # Прилагательное
        'V': wordnet.VERB,   # Глагол
        'N': wordnet.NOUN,   # Существительное
        'R': wordnet.ADV,    # Наречие
        'S': wordnet.ADJ_SAT # Прилагательное (сателлит)
    }
    
    # Берем первую букву тега
    first_char = treebank_tag[0].upper()
    
    # Возвращаем соответствующий тег WordNet или NOUN по умолчанию
    return tag_mapping.get(first_char, wordnet.NOUN)

def lemmatize_with_pos(text, remove_stopwords=True, keep_pos_tags=False):
    # Токенизация и получение POS-тегов
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    
    # Лемматизация с учетом POS
    processed = []
    for word, tag in pos_tags:
        
        if remove_stopwords and word.lower() in stop_words:
            continue
        
        wordnet_pos = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
        
        # Обработка особых случаев
        if lemma.lower() == "'s":
            lemma = 'is'
        elif lemma.lower() == "'re":
            lemma = 'are'
        elif lemma.lower() == "'m":
            lemma = 'am'
            
        processed.append((lemma, tag) if keep_pos_tags else lemma)
    
    return processed if keep_pos_tags else ' '.join(processed)

In [8]:
# Предположим, у нас есть DataFrame с текстовым столбцом 'text'
tqdm.pandas() 
df = pd.DataFrame({
    'id': [1, 2, 3],
    'text': [
        "The striped bats are hanging on their feet",
        "She's running quickly in the park",
        "I'm feeling happier today than yesterday"
    ]
})

# Применяем лемматизацию к столбцу 'text'
df['lemmatized_text'] = df['text'].progress_apply(lemmatize_with_pos)

# Результат
print(df)

100%|██████████| 3/3 [00:01<00:00,  2.07it/s]

   id                                        text  \
0   1  The striped bats are hanging on their feet   
1   2           She's running quickly in the park   
2   3    I'm feeling happier today than yesterday   

                 lemmatized_text  
0          striped bat hang foot  
1            is run quickly park  
2  am feel happy today yesterday  


In [9]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [11]:
corpus_lemm = data['text'].progress_apply(lemmatize_with_pos)

100%|██████████| 159292/159292 [10:15<00:00, 258.84it/s]


In [12]:
corpus_lemm

0         Explanation edits make username Hardcore Metal...
1         D'aww ! match background colour am seemingly s...
2         Hey man , am really try edit war . is guy cons...
3         `` ca n't make real suggestion improvement - w...
4                  , sir , hero . chance remember page is ?
                                ...                        
159287    `` : : : : : second time ask , view completely...
159288    ashamed horrible thing put talk page . 128.61....
159289    Spitzer Umm , theres actual article prostituti...
159290    look like actually put speedy first version de...
159291    `` ... really n't think understand . come idea...
Name: text, Length: 159292, dtype: object

<b>Разобьём данные на выборки</b>

In [13]:
X_train, X_test, y_train, y_test = train_test_split(corpus_lemm, data['toxic'], 
                                                    test_size=0.2,
                                                    random_state=42)

In [14]:
print(f"Размер тренировочной корпуса: {len(X_train)}")
print(f"Размер тестового корпуса: {len(X_test)}")

Размер тренировочной корпуса: 127433
Размер тестового корпуса: 31859


<b>Проведём векторизацию корпусов с помощью TfidfVectorizer и удалим стоп-слова.</b>

In [15]:
tf_idf_vec = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_words,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [16]:
X_train_vec = tf_idf_vec.fit_transform(X_train)

In [17]:
X_test_vec = tf_idf_vec.transform(X_test)

In [18]:
print(f"Размер тренировочного датасета: {X_train_vec.shape}")
print(f"Размер тестового датасета: {X_test_vec.shape}")

Размер тренировочного датасета: (127433, 41337)
Размер тестового датасета: (31859, 41337)


## Обучение

### Logistic Regression

<b>Сначала обучим логистическую регрессию. Будем подбирать гиперпараметр регуляризации C с помощью RandomizedSearchCV.</b>

In [19]:
parameters = {'C': np.linspace(10, 20, num = 5, endpoint = True),
             'max_iter': [1000]}
lrm = LogisticRegression(random_state=42)
clf = RandomizedSearchCV(lrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
clf.fit(X_train_vec, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END ..............................C=10.0, max_iter=1000; total time= 1.3min
[CV] END ..............................C=10.0, max_iter=1000; total time=  51.8s
[CV] END ..............................C=10.0, max_iter=1000; total time= 1.5min
[CV] END ..............................C=10.0, max_iter=1000; total time= 1.2min
[CV] END ..............................C=10.0, max_iter=1000; total time= 1.5min
[CV] END ..............................C=12.5, max_iter=1000; total time= 1.1min
[CV] END ..............................C=12.5, max_iter=1000; total time= 1.4min
[CV] END ..............................C=12.5, max_iter=1000; total time= 1.4min
[CV] END ..............................C=12.5, max_iter=1000; total time= 1.3min
[CV] END ..............................C=12.5, max_iter=1000; total time= 1.2min
[CV] END ..............................C=15.0, max_iter=1000; total time= 1.5min
[CV] END ..............................C=15.0, max_iter=1000; total time= 1.3min
[CV] END ...................

RandomizedSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
                   n_jobs=-1,
                   param_distributions={'C': array([10. , 12.5, 15. , 17.5, 20. ]),
                                        'max_iter': [1000]},
                   scoring='f1', verbose=2)

In [20]:
print(f"Лучший показатель f1 на кросс-валидации : {clf.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf.best_params_}")

Лучший показатель f1 на кросс-валидации : 0.771
Параметр регуляризации для лучшей модели: {'max_iter': 1000, 'C': 17.5}


<b>F1 у нашей логистической регрессии 0.773, всё отлично.</b>

### LinearSVC

In [21]:
parameters = {'C': np.linspace(1, 31, num = 7, endpoint = True)}
lsvcm = LinearSVC(max_iter = 1000)
clf_lsvc = GridSearchCV(lsvcm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
clf_lsvc.fit(X_train_vec, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV] END ..............................................C=1.0; total time=   0.7s
[CV] END ..............................................C=1.0; total time=   0.8s
[CV] END ..............................................C=1.0; total time=   0.9s
[CV] END ..............................................C=1.0; total time=   0.9s
[CV] END ..............................................C=1.0; total time=   0.8s
[CV] END ..............................................C=6.0; total time=   2.2s
[CV] END ..............................................C=6.0; total time=   2.2s
[CV] END ..............................................C=6.0; total time=   2.3s
[CV] END ..............................................C=6.0; total time=   2.2s
[CV] END ..............................................C=6.0; total time=   2.3s
[CV] END .............................................C=11.0; total time=   3.2s
[CV] END ........................................

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END .............................................C=21.0; total time=   6.2s


/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END .............................................C=21.0; total time=   6.4s


...

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END .............................................C=31.0; total time=   6.6s


/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END .............................................C=31.0; total time=   6.8s


GridSearchCV(cv=5, estimator=LinearSVC(), n_jobs=-1,
             param_grid={'C': array([ 1.,  6., 11., 16., 21., 26., 31.])},
             scoring='f1', verbose=2)

In [22]:
print(f"Лучший показатель f1 на кросс-валидации : {clf_lsvc.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf_lsvc.best_params_}")

Лучший показатель f1 на кросс-валидации : 0.775
Параметр регуляризации для лучшей модели: {'C': 1.0}


<b>Метрика f1 также больше 0.75, а именно она составляет 0.776.</b>

<b>Судя по метрике f1, выигрывает LinearSVC, выберем именно её для итогового тестирования.</b>

In [23]:
predictions = clf_lsvc.best_estimator_.predict(X_test_vec)
f1 = f1_score(y_test, predictions)
print(f"Показатель f1 на тестовой выборке: {f1:.3f}")

Показатель f1 на тестовой выборке: 0.778


## Выводы

* Успешно загрузили и обработали данные (лемматизация, указание стоп-слов и т.д.)
* Лемматизация была проведена с помощью WordNetLemmatizer
* Привели корпус к вектору с помощью TfidfVectorizer

<b>Затем обучили две модели: LogisticRegression и LinearSVC</b>
* У LogisticRegression на тренировочной выборке f1 составляет 0.773
* У LinearSVC на тренировочной выборке f1 составляет 0.776

<b>Подбор гиперпараметров осуществлялся с помощью RandomizedSearchCV и GridSearchCV.</b>


<b>По результатам f1, выбрали LinearSVC, у которой на тестовой выборке f1 составил 0.779.</b>